# Pregunta 33

¿Dónde está la referencia geográfica más repetida en la tierra de toda la Wikipedia Español?


In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/30 08:37:38 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/30 08:37:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/30 08:37:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
geo_tags = sqlContext.read.csv('./Wikipedia_dataset_2c2021/geo_tags.csv', 
                                    header=True, inferSchema=True, multiLine=True)
geo_tags_rdd = geo_tags.rdd

## Pasos

1. Filtro los contenidos que se encuentran en la tierra.
2. Para cada tupla (latitud, longitud) cuento la cantidad de veces que se repite en los articulos de Wikipedia.
3. Busco la referencia geografica (tupla (lat,long) ) que tenga mayor numero de articulos.

In [4]:
geo_tags_rdd.filter(lambda x: x.gt_globe == 'earth')\
            .map(lambda x: ( (x.gt_lat, x.gt_lon), 1) ).reduceByKey(lambda x,y: x+y)\
            .reduce(lambda a,b: a if a[1] > b[1] else b)

((-30.5, -64.5), 255)

# Conclusion 

La referencia geografica de la tierra que mas se repite en el database de Wikipedia en Español es:

| | |
|---|---|
| Latitud | -30.5 | 
| Longitud | -64.5 |
| Planeta  | Tierra

In [5]:
geo_tags_rdd.filter(lambda x: x.gt_lat == -30.5 and x.gt_lon == -64.5).first()

Row(gt_id=135261965, gt_page_id=5864596, gt_globe='earth', gt_primary=0, gt_lat=-30.5, gt_lon=-64.5, gt_dim=1000, gt_type=None, gt_name=None, gt_country=None, gt_region=None)

Lastimosamente no sabemos a que pais o region pertenece.